# Capstone Project 1 - Milestone Report

## Project Overview

**Problem**: Companies spend tremendous amounts of money on marketing in order to drive sales, but not all investments bring results. Even with the significant advancements in the ability to target the prospective customers, the targeting of the marketing campaigns is still not very precise and substantial amounts of money are still spent on marketing to the wrong audience. At the same time, the companies are now collecting more data about their customers than ever, which gives an opportunity to use that data for better targeting and profit growth.

Telemarketing remains one of the most popular marketing vehicles. The customers are usually categorized by the age group, gender, level of income and so on. Based on this preliminary targeting, telemarketers “cold call” potential customers and try to filter out those who may be interested in the product. Naturally, a lot of telemarketers’ time is wasted on calling people who have no interest in what they offer. A better targeting, hence, presents a significant opportunity to increase the effect of the telemarketing campaigns.

**Client**: Though this analysis is focused on identifying the customers who are the most likely to subscribe for bank deposits, the classification approach employed in this project can be used by any company that uses telemarketing. Better prospective customer classification will allow companies to more effectively use their telemarketers’ time and get more return on their marketing investment.

**Data**: The data for this project are sourced from the UCI Machine Learning repository and represents the results of direct marketing campaigns (phone calls) of a Portuguese banking institution. The data set has 45,211 instances and 20 features (input variables), stored in a .csv file.

http://archive.ics.uci.edu/ml/datasets/Bank+Marketing

**Method**: The expected result of the project is to predict if a customer subscribes for a deposit offer or will eventually decline it. Hence, a number of classification methods will be used, including Logistic Regression and Decision Trees.

**Deliverables**: The outcome of the project will be presented in a form of Jupyter notebook, as well as the PowerPoint presentation.

## General Information on the Data

**Overview**: The data is related to the direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls. Often, more than one contact to the same client was required, in order to access if the product (bank term deposit) would be (or not) subscribed.

This dataset is based on "Bank Marketing" UCI dataset (http://archive.ics.uci.edu/ml/datasets/Bank+Marketing). The data are enriched by the addition of five new social and economic features/attributes (national wide indicators from a ~10M population country), published by the Banco de Portugal and publicly available at: https://www.bportugal.pt/estatisticasweb. The binary classification goal is to predict if the client will subscribe a bank term deposit (variable y).

**Number of Instances**: 41,188

**Number of Attributes**: 20+ output attributes

[Detailed description of data](Data_description.txt)

## Exploratory Analysis of the Data

I looked at the overall characteristics of the dataset, starting with the dataset shape, number and types of variables, and the overall distribution of the numerical variables. Before doing the analysis, I dropped the duplicates from the dataset. By dropping the duplicates, I lost 12 observations out of 41,188, which was very little. After that, I had 21 variables (10 numeric and 11 string) and 41,176 non-duplicate rows. There were no missing values. All string variables were categorical, so they needed to be converted accordingly.

### Categorical Variables

The data didn't show any unusual distributions. There were some sparse or similar classes, which I combined into a single category to simplify the prediction models. I also dropped the unknown categories, as they represented only a small fraction of each variable's values. 

According to the data, most of the respondents were contacted during the summer months, with more than 30% of all contacts happening in May. The month of the contact can have a substantial impact on the desire to subscribe for a deposit (e.g., many people may be receiving salary bonuses at the end of the calendar year, which could be a good time to contact them about the deposit). This skewness of the previous campaigns' efforts towards summer may significantly impact the outcomes of the campaigns.

More than 86% of respondents were never covered by the previous marketing campaigns. Hence, it may make sense to divide the respondents into new customers (with 'nonexistent' status) and existing customers ('failure' or 'success' status). Within the existing customers' subset, it may be interesting to look at the predictors of the campaign success for the previously failed and successful campaigns separately - the existing customers may be more willing to open a deposit than someone who never heard of the bank before.

Additionally, only ~11% of the respondents to the current campaign have actually subscribed for a deposit, which makes the overall dataset highly imbalanced and potentially requires application of special methods to compensate for it.

### Numeric Variables

There were no obvious errors in the data. There were some prominent peaks in the distributions of the economic indicators, but it wasn't clear what was causing those peaks. There was a high likelihood that those variables will not have high predictive power for our target variable.

The number of days since the previous campaign was highly skewed because the number of days for the respondents not contacted previously were stated as 999. The variables 'campaign' and 'previous' were heavily skewed towards zero, since the majority of the respondents were never contacted with the campaign before. The variable 'duration' needed to be dropped before we start building any predictive models, as mentioned before.

The 'pdays' variable had 3,812 entries mistakenly labeled with '999'. I dropped the missing values, as the remaining data was sufficient to do further statistically significant analysis.

The detailed exploratory analysis can be found [here](Capstone_project_1.ipynb).

## Statistical Analysis

### Approach

First, I looked at the heatmap of the numeric variables to identify how they correlate, which were mostly the variables showing the current strength of economy. The economic indicators demonstrated a strong correlation with each other. Additionally, the 'previous' and 'pdays' variables demonstrated a strong negative correlation, as expected, since the largest value of 'pdays' (999) corresponds to the smallest value of 'previous' (0).

I tested the statistical significance of the strongly correlated variables representing the state of economy, i.e. of the below pairs using permutation:

- 'emp.var.rate' - 'cons.price.idx'
- 'emp.var.rate' - 'euribor3m'
- 'emp.var.rate' - 'nr.employed'
- 'cons.price.idx' - 'euribor3m'
- 'cons.price.idx' - 'nr.employed'
- 'euribor3m' - 'nr.employed'

Additionally, I checked if the categorical variables were independent of each other using a chi-square test with a 95% level of confidence. I checked the following pairs (it didn't make much sense to do a chi-square test for days of the week or months, and the dependence/independence of the outcome of the previous campaign should have been checked by a more comprehensive method, like logistic regression).

- 'job' - 'marital'
- 'job' - 'education'
- 'job' - 'housing'
- 'job' - 'loan'
- 'job' - 'contact'
- 'marital' - 'education'
- 'marital' - 'housing'
- 'marital' - 'loan'
- 'marital' - 'contact'
- 'education' - 'housing'
- 'education' - 'loan'
- 'education' - 'contact'
- 'housing' - 'loan'
- 'housing' - 'contact'
- 'loan' - 'contact'  

Finally, I performed a two-sample z-test to evaluate if the groups within our categorical variables differed significantly in whether they subscribe for the deposit or not. To do that, I grouped different levels of each categorical variable into two groups and calculated the z-score and the p-value. The null hypothesis for the test was that there's no difference between those two groups with respect to the subscription to the deposit. 

The detailed statistical analysis can be found [here](Capstone_project_1_stat_analysis.ipynb).

### Conclusions

1) All correlations between the pairs of the numeric variables are statistically significant, since the p-values are very close to zero. For two of the pairs, though - namely, 'cons.price.idx' - 'euribor3m' and 'cons.price.idx' - 'nr.employed' - the correlation coefficient isn't high. We can potentially combine these variables into one weighted variable that would represent the state of the economy.

2) Bootstrapping of the mean age of the dataset demonstrated that the mean of our dataset is very close to the mean of the sampling distribution of the bootstrapped samples' means. Hence, our dataset represents the overall population well, and there's no skew towards any particular age.

3) Based on the results of the chi-squared tests, we conclude that:

- Job level depends on the level of education
- Job level is independend from the absence or presence of a housing loan
- Job level depends on the marital status
- Job level is independend from the absence or presence of a personal loan
- Job level depends on the type of contact
- Marital status depends on the level of education
- Marital status is independend from the absence or presence of a housing loan
- Marital status is independend from the absence or presence of a personal loan
- Marital status depends on the type of contact
- Level of education is independend from the absence or presence of a housing loan
- Level of education is independend from the absence or presence of a personal loan
- Level of education depends on the type of contact
- Absence or presence of a housing loan depends on the absence or presence of a personal loan
- Absence or presence of a housing loan depends on the type of contact
- Absence or presence of a personal loan is independent from the type of contact

4) Based on the results of the two-sample z-test, we conclude that:

- The subscription to deposit is different for different levels of salary
- The subscription to deposit is different for different marital statuses
- The subscription to deposit is different for different levels of education
- The subscription to deposit is different for people with and without a housing loan
- The subscription to deposit is the same for people with and without a personal loan
- The subscription to deposit is different for different modes of contact
- The subscription to deposit is different for different seasons of the year
- The subscription to deposit is the same for the first and the second half of the week